## Подготовка данных
В `list_attr_celeba.txt`находится множество данных, которые можно использовать для различных задач. Я выбрал классифицировать фотографии на содержащие улыбку и не содержащие. 
Также необходимо преобразовать изображения из `jpg` формата в вектора и сохранить в удобном формате. 

In [1]:
import pandas as pd
import numpy as np
import tqdm

from PIL import Image

In [2]:
attr = pd.read_csv("../data_storage/list_attr_celeba.txt", delim_whitespace=True, header=1)

In [3]:
attr.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [4]:
smiling = attr['Smiling'].dropna()

Для удобства перевожу метки из -1..1 в 0..1

In [5]:
smiling = (smiling + 1 ) / 2
smiling = smiling.dropna()
print(smiling.unique())

[1. 0.]


In [6]:
## Так как у меня мало памяти, ограничусь 1000 картинок и возьму среднее по RGB каналам, а также центр 50 на 50
dataset = []
for img_index, label in tqdm.tqdm(zip(smiling.index[:1001], smiling[:1001])):
    img = np.asarray(Image.open(f'../data_storage/img_celeba/{img_index}'))
    dataset.append(np.append(img[img.shape[0] // 2 : img.shape[0] // 2 + 50, img.shape[1] // 2: img.shape[1] // 2 + 50,:].mean(-1)/255,label))

1001it [00:07, 130.37it/s]


In [7]:
with open('../data_storage/train_data.csv','w') as f:
    for d in tqdm.tqdm(dataset[:900]):
        f.write(','.join(["{:.3f}".format(x) for x in d]).strip() + '\n')
with open('../data_storage/test_data.csv','w') as f:
    for d in tqdm.tqdm(dataset[901:]):
        f.write(','.join(["{:.3f}".format(x)  for x in d]).strip() + '\n')
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 820.40it/s]


## Загрузка данных и обучение модели

In [8]:
train_data = pd.read_csv('../data_storage/train_data.csv',header=None).dropna()
test_data = pd.read_csv('../data_storage/test_data.csv',header=None).dropna()

In [9]:
X_train, y_train = train_data.drop(2500,axis=1), train_data[2500].astype(int)
X_test, y_test = test_data.drop(2500,axis=1), test_data[2500].astype(int)

In [16]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, min_samples_split=1 )

In [17]:
classifier.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=1, n_estimators=1000)

In [18]:
classifier.score(X_test, y_test)

0.54

In [52]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier(n_estimators=200, learning_rate=1e-3,max_depth=4, random_state=0,
                                        max_features='sqrt')

In [53]:
classifier.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.001, max_depth=4,
                           max_features='sqrt', n_estimators=200,
                           random_state=0)

In [54]:
classifier.score(X_test, y_test)

0.56

## Получилось не очень, но немного лучше, чем случайное предсказание. Тк попросили классический МЛ, а я не совсем знаю, что делать с большими картинками в таком случае, оставлю так